In [90]:
import pandas as pd
import numpy as np
import glob
import os
from scipy import stats
import tensorflow as tf
import math
import random
import matplotlib.pyplot as plt

fps = 120
moving_avg_len = 10
filter_value = 20
length = 150
threshold_pattern=0.4
#10.18. 목



def readFileData(file):
    column_names = ['wmx1', 'wmy1', 'wmz1']

    data = pd.read_csv(file, skiprows = 1 , names = column_names)
 
    wx = data["wmx1"]
    wy = data["wmy1"]
    wz = data["wmz1"]

    records=movingavg(wx,wy,wz)

    return np.dstack([wx,wy,wz])[0]

def readData(directory):
    records = []
    labels = []
    allFiles = glob.glob("*.csv")
    for i,file in enumerate(allFiles):
        fileName = os.path.basename(file)
        print("file name is", fileName,"\nnum is",i)
        (name, ext) = os.path.splitext(fileName)
        print("name is ",name,"ext is",ext)
        parts = name.split("_")
        if (True):
            label = parts[1] #'circle', 'triangle', 'X'
            fileData = readFileData(file)
            records.append(fileData)
            #print("file data is ",fileData)
            labels.append(label)

    print("* labels : ", labels)
    return (records, labels)

def movingavg(x,y,z):
    x_avg = np.zeros(moving_avg_len)/moving_avg_len
    y_avg = np.zeros(moving_avg_len)/moving_avg_len
    z_avg = np.zeros(moving_avg_len)/moving_avg_len
    x_avg = np.convolve(x,x_avg,'same')
    y_avg = np.convolve(y,y_avg,'same')
    z_avg = np.convolve(z,z_avg,'same')
    
    return [x_avg,y_avg,z_avg]

def splitData(records):##sungshil
    
    term=fps // 24
    record_sum = []
    record = []
    record_all = []
    sum_now = 0
    sum_pre = 0
    ct = 0
    start = 0
    end = 0
    num_pattern=[]
    for t in range(len(records)):
        ct = 0
        record_one=[]
        for i in range((len(records[t])//(term))-100):
            sum = 0
            record_x = np.empty(shape=[1],dtype=float)
            record_y = np.empty(shape=[1],dtype=float)
            record_z = np.empty(shape=[1],dtype=float)
            for j in range(fps//2):
                record_x = np.append(record_x,records[t][term*i+j][0])
                record_y = np.append(record_y,records[t][term*i+j][1])
                record_z = np.append(record_z,records[t][term*i+j][2])
            for k in range(fps//2 - 2):
                sum = sum+(record_x[k]-record_x[k+1])**2+(record_y[k]-record_y[k+1])**2+(record_z[k]-record_z[k+1])**2

            sum_pre = sum_now
            sum_now = sum
            record_sum.append(sum)
            if (sum_pre < threshold_pattern and sum_now > threshold_pattern):
                start = term * i
            if (sum_pre > threshold_pattern and sum_now < threshold_pattern):
                end = term * i  
                s = []
                if (end-start)>120:
                    ct = ct + 1
                    #print(">> start : ",start)
                    #print(" >> end : ",end,", step : " ,ct, ", length of end-start : ",(end-start))
                    for k in range(end-start):
                        #t=(records[0][term*i+j+k])
                        #print("t is",t)
                        s.append(records[t][start+k])
                        #print("s is ",s)
                    record_one.append(s)
        record_all.append(record_one)
        print("number of patterm : ",ct)
        num_pattern.append(ct)
        #print("average is",np.average(record_sum))
        #plt.plot(record_sum[1500:2000])
 
    return (record_all,num_pattern)

def getRecordsMaxLength(records):
    maxLen = 0
    for record in records:
        if (len(record) > maxLen):
            maxLen = len(record)
        
    return maxLen

def shortenRecordsLen(records, length):
    ret1 = []
    #print("records[]",len(records))
    for num in range(len(records)):
        ret2 = []
        for index in range(len(records[num])):
            s = []
            if (len(records[num][index])>length):
                record = records[num][index]
                if len(record) < 1000:
                    for i in range(length):
                        ret3=[]                      
                        t=i*(len(record)/length)
                        if t%1 == 0:
                            s.append(record[int(t)])
                        else:
                            a = t//1
                            s1 = t-a
                            s2 = a+1-t
                            for j in range(3):
                                ret3.append( s1*record[int(a)][j] + s2*record[int(a)][j] )
                            s.append(ret3)
                ret2.append(s)
        ret1.append(ret2)

    return ret1
def labeling(records, num_pattern):
    labels = []
    '''recods 합쳐진 다음에 num_pattern로 제스쳐 개수 계산하지 말고 records length로 바꾸기 '''
    for i in range(num_pattern[0] + num_pattern[1] + num_pattern[2]):
        if i < num_pattern[0] :
            labels.append('circle')
        elif i < num_pattern[0] + num_pattern[1]:
            labels.append('triangle')
        else :labels.append('X')
    print("labelling - labels length : ", len(labels))
    return labels
            
def weight_variable(shape):#가중치 생성
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):#편향 생성
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):#각 채널마다 활성곱 수행후 ReLu활성화 함수로 pass
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):#생성된 convolution층에 pooling 적용
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

In [99]:
(records,labels) = readData("dr")
print("**len_record : ",len(records))
records,nums_pattern = splitData(records)

file name is Converted_circle_Sua_1012.csv 
num is 0
name is  Converted_circle_Sua_1012 ext is .csv
file name is Converted_triangle_Sua_1012.csv 
num is 1
name is  Converted_triangle_Sua_1012 ext is .csv
file name is Converted_X_Sungshil_1012.csv 
num is 2
name is  Converted_X_Sungshil_1012 ext is .csv
* labels :  ['circle', 'triangle', 'X']
**len_record :  3
number of patterm :  116
number of patterm :  79
number of patterm :  98


In [104]:
len_label=len(records)
print(len_label)
shortened_records = shortenRecordsLen(records, length)
print(len(shortened_records[1])) 
nums_gesture = [len(shortened_records[0]), len(shortened_records[1]), len(shortened_records[2])]
labels = labeling(records, nums_gesture)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

3
68
labelling - labels length :  282


In [88]:
print(labels)
print(len(labels))

[[0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 

In [6]:
for i in range(len(records[1])):
    plt.plot(records[1][i])

In [49]:
print(len(records[0])," | ", len(records[1])," | ", len(records[2]))
print(len(records[0][0])," | ", len(records[1][0])," | ", len(records[2][0]))
flattened_records = records.ravel()
#reshaped_records = flatten_records.reshape(282,150, 3) #recoed를 1차원으로 reshape
#print(reshaped_records)

116  |  68  |  98
150  |  150  |  150
[array([-0.1767024, -0.3798141,  0.3363059]), [-0.1766218, -0.3795925, 0.3366926], [-0.1769349, -0.3792469, 0.3369359], [-0.176872, -0.3789514, 0.3373609], [-0.1767111, -0.3786908, 0.3377913], [-0.176334, -0.3783776, 0.3384521], [-0.1760631, -0.3781414, 0.33878709999999995], [-0.175324, -0.37814380000000003, 0.33922040000000003], [-0.1754771, -0.37720570000000003, 0.340052], [-0.1749683, -0.3768491, 0.3406613], [-0.1742271, -0.37749299999999997, 0.3404283], [-0.1745379, -0.3765695, 0.3411479], [-0.1743856, -0.37649469999999996, 0.3412591], [-0.17419469999999998, -0.3762939, 0.34144179999999996], [-0.1736866, -0.37700159999999994, 0.3409831], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.34101609999999993], [-0.1735362, -0.3769652, 0.341

In [ ]:
train_test_split = np.random.rand(len(reshaped_records)) < 0.70
print(train_test_split)
train_x = reshaped_records[train_test_split]
print("len(train_x) : ",len(train_x))
print(train_x)
train_y = labels[train_test_split]
test_x = reshaped_records[~train_test_split]
test_y = labels[~train_test_split]


In [ ]:
input_height = 1
input_width = 200
num_labels = 1
num_channels = 3 

batch_size = 10
kernel_size = 60 #??
depth = 60
num_hidden = 1000

learning_rate = 0.0001
training_epochs = 8

total_batches = train_x.shape[0] // batch_size

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
# input data
Y = tf.placeholder(tf.float32, shape=[None,num_labels])
# output data

c = apply_depthwise_conv(X,kernel_size,num_channels,depth) #1d convolution
p = apply_max_pool(c,20,2) #1d Max-Pooling
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10) #1d convolution

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)
# softmax

In [ ]:
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
with tf.Session() as session: 
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        cost_history = np.empty(shape=[1],dtype=float)
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        print ("Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ", session.run(accuracy, feed_dict={X: train_x, Y: train_y}))
    print ("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))